In [ ]:
import os
import shutil
import pandas as pd
import subprocess
from ultralytics import YOLO

# Set parameters
data_yaml = '/home/student/Desktop/current_model_ai/data.yaml'
project_dir = '/home/student/Desktop/current_model_ai/folder_train/folds_retrain/versions'
model_name = 'yolov8x.pt'  # Change this to test other models
name = model_name.replace('.pt', '')
output_dir = os.path.join(project_dir, name)
csv_path = os.path.join(project_dir, f'{name}__results.csv')

# Training settings
epochs = 100
imgsz = 640
batch = 16

# Clean previous output
if os.path.exists(output_dir):
    shutil.rmtree(output_dir)

print(f"Training model: {model_name}")
model = YOLO(model_name)

# Train
model.train(
    data=data_yaml,
    epochs=epochs,
    imgsz=imgsz,
    batch=batch,
    project=project_dir,
    name=name,
    augment=True
)

# Validate
metrics = model.val()

# Save results: only Model, mAP@0.5, mAP@0.5:0.95, Recall
new_result = {
    'Model': name,
    'mAP@0.5': metrics.box.map50,
    'mAP@0.5:0.95': metrics.box.map,
    'Recall': metrics.box.mr  # no parentheses!
}

df = pd.DataFrame([new_result])
df.to_csv(csv_path, index=False)
print(f"Results saved to {csv_path}")


In [8]:
import os
import pandas as pd
import matplotlib.pyplot as plt

# Update this to your local folder
results_dir = '/home/student/Desktop/current_model_ai/folder_train/folds_retrain/versions'
plot_output_dir = os.path.join(results_dir, 'comparison_plots')
os.makedirs(plot_output_dir, exist_ok=True)

# Find all model result CSVs
result_files = [f for f in os.listdir(results_dir) if f.endswith('__results.csv')]

# Load and combine them
all_results = []
for file in result_files:
    df = pd.read_csv(os.path.join(results_dir, file))
    all_results.append(df)

combined_df = pd.concat(all_results, ignore_index=True)

# Ensure the models are in a specific order
model_order = ['yolov8n', 'yolov8s', 'yolov8m', 'yolov8l', 'yolov8x']
combined_df['Model'] = pd.Categorical(combined_df['Model'], categories=model_order, ordered=True)
combined_df = combined_df.sort_values('Model')

# Save combined data for reference
combined_df.to_csv(os.path.join(results_dir, 'combined_model_results.csv'), index=False)

# Plot mAP@0.5
plt.figure(figsize=(10, 6))
plt.bar(combined_df['Model'], combined_df['mAP@0.5'], color='skyblue')
plt.title('mAP@0.5 Comparison')
plt.xlabel('Model Version')
plt.ylabel('mAP@0.5')
plt.grid(True)
plt.tight_layout()
plt.savefig(os.path.join(plot_output_dir, 'mAP50_comparison.png'))
plt.close()

# Plot mAP@0.5:0.95
plt.figure(figsize=(10, 6))
plt.bar(combined_df['Model'], combined_df['mAP@0.5:0.95'], color='orange')
plt.title('mAP@0.5:0.95 Comparison')
plt.xlabel('Model Version')
plt.ylabel('mAP@0.5:0.95')
plt.grid(True)
plt.tight_layout()
plt.savefig(os.path.join(plot_output_dir, 'mAP5095_comparison.png'))
plt.close()

# Plot Recall
plt.figure(figsize=(10, 6))
plt.bar(combined_df['Model'], combined_df['Recall'], color='green')
plt.title('Recall Comparison')
plt.xlabel('Model Version')
plt.ylabel('Recall')
plt.grid(True)
plt.tight_layout()
plt.savefig(os.path.join(plot_output_dir, 'Recall_comparison.png'))
plt.close()

# Identify best model by highest mAP@0.5:0.95
best_model_row = combined_df.loc[combined_df['mAP@0.5:0.95'].idxmax()]
best_model_info = f"Best model for your system: {best_model_row['Model']} with mAP@0.5:0.95 = {best_model_row['mAP@0.5:0.95']:.4f}"
print(best_model_info)

# Save summary
with open(os.path.join(results_dir, 'best_model_summary.txt'), 'w') as f:
    f.write(best_model_info)



Best model for your system: yolov8n with mAP@0.5:0.95 = 0.1594


In [9]:
import os
import pandas as pd
import matplotlib.pyplot as plt

# Path to the base directory containing model folders
base_dir = '/home/student/Desktop/current_model_ai/folder_train/folds_retrain/versions'
model_names = ['yolov8n', 'yolov8s', 'yolov8m', 'yolov8l', 'yolov8x']

# Metrics to compare across models
metrics_to_plot = [
    'train/box_loss', 'train/cls_loss',
    'val/box_loss', 'val/cls_loss', 'val/dfl_loss',
    'metrics/precision(B)', 'metrics/recall(B)',
    'metrics/mAP50(B)', 'metrics/mAP50-95(B)'
]

# Output folder for plots
plot_output_dir = os.path.join(base_dir, 'detailed_comparison_plots')
os.makedirs(plot_output_dir, exist_ok=True)

# Load each model's results.csv
model_data = {}
for model in model_names:
    csv_path = os.path.join(base_dir, model, 'results.csv')
    if os.path.exists(csv_path):
        df = pd.read_csv(csv_path)
        model_data[model] = df

# Plot each metric over epochs
for metric in metrics_to_plot:
    plt.figure(figsize=(10, 6))
    for model, df in model_data.items():
        if metric in df.columns:
            plt.plot(df['epoch'], df[metric], label=model)
    plt.title(f'{metric} Comparison Over Epochs')
    plt.xlabel('Epoch')
    plt.ylabel(metric)
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    filename = metric.replace('/', '_').replace('(', '').replace(')', '') + '.png'
    plt.savefig(os.path.join(plot_output_dir, filename))
    plt.close()


In [7]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os

# Load CSV
csv_path = '/home/student/Desktop/current_model_ai/hyperparameter_tuning_results.csv'
df = pd.read_csv(csv_path)

# Rename for consistency
df = df.rename(columns={
    'Batch Size': 'Batch',
    'mAP50': 'mAP@0.5',
    'mAP50-95': 'mAP@0.5:0.95'
})
df['Batch'] = df['Batch'].astype(str)

# Create output directory for plots
plot_dir = os.path.join(os.path.dirname(csv_path), 'grouped_bar_plots')
os.makedirs(plot_dir, exist_ok=True)

# Metrics to plot
metrics = ['Recall', 'Precision', 'mAP@0.5', 'mAP@0.5:0.95']

# Plot grouped bars for each metric
for metric in metrics:
    plt.figure(figsize=(10, 6))
    sns.barplot(data=df, x='Epochs', y=metric, hue='Batch')
    plt.title(f'{metric} Across Epoch and Batch Size Combinations')
    plt.xlabel('Epochs')
    plt.ylabel(metric)
    plt.legend(title='Batch Size')
    plt.grid(True)
    plt.tight_layout()
    filename = f"{metric.replace('@', '_at_').replace('.', '_')}_grouped_bar.png"
    plt.savefig(os.path.join(plot_dir, filename))
    plt.close()

# Find best combination based on highest mAP@0.5:0.95
best_row = df.loc[df['mAP@0.5:0.95'].idxmax()]
best_batch = best_row['Batch']
best_epochs = best_row['Epochs']
best_map = best_row['mAP@0.5:0.95']

# Save summary
summary_path = os.path.join(os.path.dirname(csv_path), 'best_model_config.txt')
with open(summary_path, 'w') as f:
    f.write(f"Best model configuration for your system:\n")
    f.write(f"Batch Size: {best_batch}\n")
    f.write(f"Epochs: {best_epochs}\n")
    f.write(f"mAP@0.5:0.95: {best_map:.4f}\n")

print("Grouped plots saved to:", plot_dir)
print("Best configuration written to:", summary_path)
print(f"\nBest configuration:\nBatch Size: {best_batch}\nEpochs: {best_epochs}\nmAP@0.5:0.95: {best_map:.4f}")




Grouped plots saved to: /home/student/Desktop/current_model_ai/grouped_bar_plots
Best configuration written to: /home/student/Desktop/current_model_ai/best_model_config.txt

Best configuration:
Batch Size: 8
Epochs: 50
mAP@0.5:0.95: 0.4110


In [12]:
#GGGAMMMEL Kan slettes snart


import os
import shutil
import pandas as pd
import subprocess
from ultralytics import YOLO
# Define the training output folder path
train_folder = "/home/student/Desktop/current_model_ai/folder_train/folds_retrain/test"

# Remove the folder if it exists
if os.path.exists(train_folder):
    shutil.rmtree(train_folder)

# Load the model
model = YOLO('yolov8s.pt')

# Train the model
model.train(
    data='/home/student/Desktop/current_model_ai/data.yaml',
    epochs=50,
    imgsz=640,
    batch=8,
    project='/home/student/Desktop/current_model_ai/folder_train/test',
    name='folds_retrain',
    augment=True,
    auto_augment='randaugment',
    erasing=0.4,
    flipud=0.1,
    translate=0.1,
    scale=0.5,
    mixup=0.2,
    mosaic=0.0

)



New https://pypi.org/project/ultralytics/8.3.144 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.133 🚀 Python-3.10.16 torch-2.6.0+cu124 CUDA:0 (NVIDIA RTX 4000 Ada Generation, 20040MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=True, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/home/student/Desktop/current_model_ai/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.1, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.2, mode=train, model=yolov8s.pt, momentum=0.937, mosaic=0.0, multi_scale=False, name=folds_retra

train: Scanning /home/student/Desktop/current_model_ai/train/labels.cache... 1184 images, 466 backgrounds, 0 corrupt: 100%|██████████| 1648/1648 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, method='weighted_average', num_output_channels=3), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))


val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 2948.5±2369.5 MB/s, size: 334.0 KB)


val: Scanning /home/student/Desktop/current_model_ai/val/labels.cache... 338 images, 489 backgrounds, 0 corrupt: 100%|██████████| 827/827 [00:00<?, ?it/s]


...

                   all        827        506      0.367      0.877      0.359       0.16



50 epochs completed in 0.207 hours.
Optimizer stripped from /home/student/Desktop/current_model_ai/folder_train/test/folds_retrain/weights/last.pt, 22.5MB
Optimizer stripped from /home/student/Desktop/current_model_ai/folder_train/test/folds_retrain/weights/best.pt, 22.5MB

Validating /home/student/Desktop/current_model_ai/folder_train/test/folds_retrain/weights/best.pt...
Ultralytics 8.3.133 🚀 Python-3.10.16 torch-2.6.0+cu124 CUDA:0 (NVIDIA RTX 4000 Ada Generation, 20040MiB)
Model summary (fused): 72 layers, 11,125,971 parameters, 0 gradients, 28.4 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 52/52 [00:04<00:00, 12.70it/s]


                   all        827        506      0.359      0.891      0.352      0.159
Speed: 0.1ms preprocess, 3.8ms inference, 0.0ms loss, 0.3ms postprocess per image
Results saved to /home/student/Desktop/current_model_ai/folder_train/test/folds_retrain


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7f67564caa10>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.048048, 

In [ ]:

# Define the training output folder path
train_folder = "/home/student/Desktop/current_model_ai/folder_train/folds_retrain"

# Remove the folder if it exists
if os.path.exists(train_folder):
    shutil.rmtree(train_folder)

# Load the model
model = YOLO('yolov8m.pt')

model.train(
    data='/home/student/Desktop/current_model_ai/data.yaml',
    epochs=50,
    imgsz=640,
    batch=8,
    project='/home/student/Desktop/current_model_ai/folder_train',
    name='folds_retrain',
    augment=True,
    auto_augment='randaugment',
    degrees=10,
    scale=0.5,
    shear=10,
    perspective=0.001,
    flipud=0.1,
    fliplr=0.5,
    hsv_h=0.015,
    hsv_s=0.7,
    hsv_v=0.4,
    mixup=0.2,
    copy_paste=0.1,
    erasing=0.3,
    patience=20,
    lr0=0.002,
    lrf=0.01
)


New https://pypi.org/project/ultralytics/8.3.144 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.133 🚀 Python-3.10.16 torch-2.6.0+cu124 CUDA:0 (NVIDIA RTX 4000 Ada Generation, 20040MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=True, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.1, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/home/student/Desktop/current_model_ai/data.yaml, degrees=10, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.3, exist_ok=False, fliplr=0.5, flipud=0.1, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.002, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.2, mode=train, model=yolov8s.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=folds_retra

train: Scanning /home/student/Desktop/current_model_ai/train/labels.cache... 1184 images, 466 backgrounds, 0 corrupt: 100%|██████████| 1648/1648 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, method='weighted_average', num_output_channels=3), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))


val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 2202.3±1613.3 MB/s, size: 334.0 KB)


val: Scanning /home/student/Desktop/current_model_ai/val/labels.cache... 338 images, 489 backgrounds, 0 corrupt: 100%|██████████| 827/827 [00:00<?, ?it/s]


...

                   all        827        506      0.364      0.864      0.354      0.149



50 epochs completed in 0.205 hours.
Optimizer stripped from /home/student/Desktop/current_model_ai/folder_train/folds_retrain/weights/last.pt, 22.5MB
Optimizer stripped from /home/student/Desktop/current_model_ai/folder_train/folds_retrain/weights/best.pt, 22.5MB

Validating /home/student/Desktop/current_model_ai/folder_train/folds_retrain/weights/best.pt...
Ultralytics 8.3.133 🚀 Python-3.10.16 torch-2.6.0+cu124 CUDA:0 (NVIDIA RTX 4000 Ada Generation, 20040MiB)
Model summary (fused): 72 layers, 11,125,971 parameters, 0 gradients, 28.4 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 52/52 [00:03<00:00, 13.25it/s]


                   all        827        506      0.358      0.866      0.356      0.156
Speed: 0.1ms preprocess, 3.6ms inference, 0.0ms loss, 0.4ms postprocess per image
Results saved to /home/student/Desktop/current_model_ai/folder_train/folds_retrain


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7f6590a24b50>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.048048, 